# History of Spark

## grep
- Tutorial: https://www.cyberciti.biz/faq/howto-use-grep-command-in-linux-unix/

## Hadoop
- MapReduce (2004)
- Hadoop @Yahoo (2006)
- "Take the processing to the data"

<img src="https://upload.wikimedia.org/wikipedia/commons/0/0e/Hadoop_logo.svg" width="60%"/>

https://phoenixnap.com/kb/hadoop-vs-spark

## [Spark](http://spark.apache.org/)
- Spark @UC Berkley (2009)
- Open Source Spark, BSD License (2010)
- Developed by AmpLab (2011)
- Databricks maintains open source Spark (2013) => Stability
- License change to Apache Commons 2.0 (2013)
- [Large Scale Sorting record](https://databricks.com/blog/2014/11/05/spark-officially-sets-a-new-record-in-large-scale-sorting.html) (2014)
- Spark 2.x (2016)
- Spark 3.x (2020)

## Languages

### [Scala](https://www.scala-lang.org/)
- Designed by Martin Odersky (2004)
- Developed by EPFL (Switzerland)

### [Java](https://docs.oracle.com/javase/8/docs/technotes/guides/language/index.html)
  - Designed by James Gosling (1995)
  - Developed by Oracle

### [Python](https://www.python.org/)
- Designed by Guido van Rossum (1991)
- Developed by Python Software Foundation

### [R](https://www.r-project.org/)
  - Designed by Ross Ihaka and Robert Gentleman (1993)
  - Developed by R Core Team

### [C#](https://docs.microsoft.com/en-us/dotnet/csharp/)
- Designed by Anders Hejlsberg (2000)
- Developed by Microsoft

### [F#](https://docs.microsoft.com/en-us/dotnet/fsharp/)
- Designed by Don Syme (2005)
- Developed by Microsoft and The F# Software Foundation

### SQL
- Designed by Donald D. Chamberlin and Raymond F. Boyce (1974)
- Developed by 	ISO/IEC

## Libraries

### SQL

### Streaming

### MLlib

### GraphX

# Data

In [1]:
from google.colab import drive
import os
from requests import get

In [2]:
!ls

sample_data


In [3]:
%%html
<iframe src="https://corpus.canterbury.ac.nz/descriptions/" width="800" height="600"></iframe>

In [4]:
drive.mount('/content/gdrive', force_remount=True)
dir = os.path.join('gdrive', 'My Drive', 'Eurostat', '05 - Data Science for Big Data')
data_dir = os.path.join(dir, 'data')

Mounted at /content/gdrive


In [5]:
def download_save(url, filename):
  res = get(url)
  if res.status_code != 200:
    print(f"Couldn't fetch data from {url}")
  else:
    csv_file = open(os.path.join(data_dir, filename), 'wb')
    csv_file.write(res.content)
    csv_file.close()

In [6]:
download_save('http://corpus.canterbury.ac.nz/resources/large.zip',
              'large.zip')

In [7]:
!cd "gdrive/MyDrive/Eurostat/05 - Data Science for Big Data/data" && unzip large.zip

Archive:  large.zip
  inflating: bible.txt               
  inflating: E.coli                  
  inflating: world192.txt            


# Hello Spark

## Pyspark Installation

In [8]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 15.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=0c1a52dbe7530ea5c8552e185fbb5a64a26be9fb21158ba46a9c4de66134595a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## Spark Context

In [9]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("Hello-Spark").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [10]:
sc

<SparkContext master=local[*] appName=Hello-Spark>

In [11]:
file_name = os.path.join(data_dir, 'bible.txt')
bible_file = sc.textFile(file_name)

In [12]:
help(bible_file)

Help on RDD in module pyspark.rdd object:

class RDD(builtins.object)
 |  RDD(jrdd, ctx, jrdd_deserializer=AutoBatchedSerializer(PickleSerializer()))
 |  
 |  A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.
 |  Represents an immutable, partitioned collection of elements that can be
 |  operated on in parallel.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      Return the union of this RDD and another one.
 |      
 |      Examples
 |      --------
 |      >>> rdd = sc.parallelize([1, 1, 2, 3])
 |      >>> (rdd + rdd).collect()
 |      [1, 1, 2, 3, 1, 1, 2, 3]
 |  
 |  __getnewargs__(self)
 |  
 |  __init__(self, jrdd, ctx, jrdd_deserializer=AutoBatchedSerializer(PickleSerializer()))
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  aggregate(self, zeroValue, seqOp, combOp)
 |      Aggregate the elements of each partition, and then the results for all
 |      the par

In [13]:
type(bible_file)

pyspark.rdd.RDD

In [14]:
print(bible_file.__doc__)


    A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.
    Represents an immutable, partitioned collection of elements that can be
    operated on in parallel.
    


In [15]:
bible_file.first()

'In the beginning God created the heaven and the earth. And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters. '

# Word Count

## Map & Reduce


In [16]:
first_line = "In the beginning God created the heaven and the earth. And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters."

In [17]:
def decompose_to_words(line):
  return line.split()

In [18]:
bible_words = bible_file.flatMap(lambda line : line.split(" "))

In [19]:
word_counts = bible_words.map(lambda word : (word, 1))

In [20]:
word_counts = word_counts.reduceByKey(lambda cumulVal, newVal : cumulVal + newVal)

In [21]:
type(bible_words)

pyspark.rdd.PipelinedRDD

## Lazy Evaluation

In [22]:
word_counts.take(4)

[('God', 2186), ('created', 36), ('And', 12163), ('earth', 326)]

In [23]:
word_counts = word_counts.sortBy(lambda kv_pair : kv_pair[1], ascending=False)

In [24]:
word_counts.take(4)

[('the', 59835), ('and', 37322), ('of', 32972), ('', 30383)]

In [25]:
word_counts.saveAsTextFile(os.path.join(data_dir, 'bible_word_count'))

## CountByValue

In [26]:
bible_file.flatMap(lambda line : line.split(" ")).countByValue()

defaultdict(int,
            {'In': 309,
             'the': 59835,
             'beginning': 68,
             'God': 2186,
             'created': 36,
             'heaven': 202,
             'and': 37322,
             'earth.': 191,
             'And': 12163,
             'earth': 326,
             'was': 4249,
             'without': 372,
             'form,': 3,
             'void;': 3,
             'darkness': 61,
             'upon': 2659,
             'face': 265,
             'of': 32972,
             'deep.': 5,
             'Spirit': 118,
             'moved': 45,
             'waters.': 25,
             '': 30383,
             'said,': 1556,
             'Let': 417,
             'there': 1799,
             'be': 6643,
             'light:': 10,
             'light.': 28,
             'saw': 508,
             'light,': 53,
             'that': 12107,
             'it': 4336,
             'good:': 33,
             'divided': 54,
             'light': 160,
             'from': 

In [27]:
sc.stop()